<a href="https://colab.research.google.com/github/sugiyama404/ReinfoceLearningForTrading/blob/main/a2c_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
from datetime import datetime
import random
import copy

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler

from google.colab import drive

drive.mount('/content/drive/')
nov_dir = 'Colab Notebooks/dataset/reinforcement_learning/'
nov_path = '/content/drive/My Drive/' + nov_dir + 'sp500_test.csv'
'''
exp_dir = 'Colab Notebooks/workspace/export/'
csv_path = '/content/drive/My Drive/' + exp_dir + 'random_test.csv'
'''

df = pd.read_csv(nov_path)
df['Date'] = pd.to_datetime(df['Date'], format = '%Y-%m-%d')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
def make_scaler(env):

    states = []
    for _ in range(env.df_total_steps):
        action = np.random.choice(env.action_space)
        state, reward, done, info = env.step(action)
        states.append(state)
        if done:
            break

    scaler = StandardScaler()
    scaler.fit(states)
    return scaler

In [3]:
class Environment:
    def __init__(self, df, initial_money=100000, mode = 'test'):

        self.df = df.dropna().reset_index()
        self.df_total_steps = len(self.df)-1
        self.initial_money = initial_money
        self.mode = mode
        self.trade_time = None
        self.trade_win = None
        self.brfore_buy_cash = None
        self.action_space = np.array([0, 1, 2])
        self.hold_a_position = None
        self.now_price = None
        self.cash_in_hand = None

        self.reset()
        
    def reset(self):

        self.trade_time = 0
        self.trade_win = 0
        self.brfore_buy_cash = 0
        self.end_step = self.df_total_steps
        self.now_step = 0
        self.hold_a_position = 0.0
        self.now_price = self.df.loc[self.now_step, 'SP500']
        self.cash_in_hand = self.initial_money

        return self._get_now_state()

    def step(self, action):

        prev_revenue = self._get_revenue()
        self.now_step += 1
        self.now_price = self.df.loc[self.now_step, 'SP500']
 
        done = (self.end_step == self.now_step)

        self._trade(action,done)
        cur_revenue = self._get_revenue()
 
        reward = cur_revenue - prev_revenue

        if self.mode == 'test':
            info = { 'cur_revenue' : cur_revenue , 'trade_time' : self.trade_time, 'trade_win' : self.trade_win }
        else:
            info = { 'cur_revenue' : cur_revenue }

        return self._get_now_state(), reward, done, info

    def _get_now_state(self):
        state = np.empty(3)
        state[0] = self.hold_a_position
        state[1] = self.now_price
        state[2] = self.cash_in_hand
        return state

    def _get_revenue(self): 
        return self.hold_a_position * self.now_price + self.cash_in_hand

    def _trade(self, action,lastorder = False):
        if lastorder:
            self.cash_in_hand += self.now_price * self.hold_a_position
            self.hold_a_position = 0
        else:
            if self.action_space[0] == action: # sell
                if self.hold_a_position != 0:
                    self.cash_in_hand += self.now_price * self.hold_a_position
                    self.hold_a_position = 0
                    if self.mode == 'test':
                        self.trade_time += 1
                        if self.cash_in_hand > self.brfore_buy_cash:
                            self.trade_win += 1 
            if self.action_space[2] == action: # buy
                if self.hold_a_position == 0:
                    buy_flag = True
                    if self.mode == 'test':
                        self.brfore_buy_cash = copy.copy(self.cash_in_hand)
                    while buy_flag:
                        if self.cash_in_hand > self.now_price:
                            self.hold_a_position += 1
                            self.cash_in_hand -= self.now_price
                        else:
                            buy_flag = False

In [4]:
class Brain:
    def __init__(self,n_action = 3):

        n_shape = 3
        self.n_action = n_action
        lr = 0.01

        common = input_ = keras.layers.Input(shape=n_shape)
        common = keras.layers.Dense(128, activation="relu")(common)

        actor = keras.layers.Dense(self.n_action, activation="softmax")(common)
        critic = keras.layers.Dense(1, activation="linear")(common)

        model = keras.Model(input_, [actor, critic])
        model.compile(optimizer=Adam(lr=lr))
        model.summary()
        self.model = model

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

In [5]:
class Actor:
    def __init__(self,model,n_action = 3):
        self.model = model
        self.n_action = n_action

    def policynetwork(self, state):
        act_p, _ = self.model(state.reshape((1,-1)))
        return np.random.choice(self.n_action, p=act_p[0].numpy())

In [6]:
class Critic:
    def __init__(self,model,n_action=3):
        self.model = model
        self.n_action = n_action
        self.gamma = 0.9
        self.alpha = 0.5
        self.beta = 0.1

    def valuenetwork(self, experiences):

        discounted_rewards = self._discounted_rewards(experiences)

        state_batch = np.asarray([e["state"] for e in experiences])
        action_batch = np.asarray([e["action"] for e in experiences])

        onehot_actions = tf.one_hot(action_batch, self.n_action)

        with tf.GradientTape() as tape:

            act_p, v = self.model(state_batch, training=True)

            selct_pai = tf.reduce_sum(onehot_actions * act_p, axis=1, keepdims=True)
            selected_action_probs = tf.clip_by_value(selct_pai, 1e-10, 1.0)

            policy_loss = self._policy_loss(selected_action_probs, discounted_rewards, v)
            value_loss = self._value_loss(discounted_rewards,v)         
            policy_entropy = self._policy_entropy(discounted_rewards,v,selected_action_probs)

            total_loss = -policy_loss + self.alpha * value_loss - self.beta * policy_entropy
            loss = tf.reduce_mean(total_loss)

        gradients = tape.gradient(loss, self.model.trainable_variables)
        self.model.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))

    def _discounted_rewards(self,experiences):
        if experiences[-1]["done"]:
            G = 0
        else:
            next_state = np.atleast_2d(experiences[-1]["next_state"])
            _, n_v = self.model(next_state)
            G = n_v[0][0].numpy()

        discounted_rewards = []
        for exp in reversed(experiences):
            if exp["done"]:
                G = 0
            G = exp["reward"] + self.gamma * G
            discounted_rewards.append(G)
        discounted_rewards.reverse()
        discounted_rewards = np.asarray(discounted_rewards).reshape((-1, 1))
        discounted_rewards -= np.mean(discounted_rewards)
        return discounted_rewards

    def _policy_loss(self, selected_action_probs, discounted_rewards, v):
        advantage = discounted_rewards - tf.stop_gradient(v)
        return tf.math.log(selected_action_probs) * advantage

    def _value_loss(self, discounted_rewards, v):
        return tf.reduce_mean((discounted_rewards - v) ** 2, axis=1, keepdims=True)

    def _policy_entropy(self, discounted_rewards, v, selected_action_probs):
        return tf.reduce_sum(tf.math.log(selected_action_probs) * selected_action_probs, axis=1, keepdims=True)

In [7]:
def play_game(env, actor,critic, scaler, episodes_times = 100, batch_size = 32, mode = 'train'):
    experiences = []
    episode_rewards = []

    for episode in range(episodes_times):
        state = env.reset()
        state = scaler.transform([state])
        state = state.flatten()
        done = False
        total_reward = 0
        step = 0

        while not done:
            
            action = actor.policynetwork(state)
            next_state, reward, done, _ = env.step(action)
            next_state = scaler.transform([next_state])
            next_state = next_state.flatten()

            total_reward += reward

            if mode == 'train':
                experiences.append({"state": state, "action": action, "reward": reward, "next_state": next_state, "done": done,})
                if len(experiences) == batch_size:
                    critic.valuenetwork(experiences)
                    experiences = []
            
            state = next_state
            step += 1

        episode_rewards.append(total_reward)
        if episode % 10 == 0:
            print("{}/{}, {} step, reward: {}".format(episode, episodes_times, step, np.round(np.mean(episode_rewards[-10:]), decimals=2)))

### 学習の実行

In [8]:
initial_money=1000000
mode = 'train'

brain = Brain()
model = brain.model

actor = Actor(model)
critic = Critic(model)

if mode == 'test':
    brain.load(f'a2c_model.h5')

env = Environment(df, initial_money = initial_money, mode=mode)
scaler = make_scaler(env)

play_game(env, actor, critic, scaler, mode = mode)

if mode == 'train':
    brain.save(f'a2c_model.h5')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 3)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 128)          512         input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 3)            387         dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 1)            129         dense[0][0]                      
Total params: 1,028
Trainable params: 1,028
Non-trainable params: 0
__________________________